In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm
from scipy import stats
from finance_byu.fama_macbeth import fama_macbeth, fm_summary

In [10]:
CRSP = pd.read_sas('crsp.sas7bdat', encoding='latin-1') #fp3v1
COMP = pd.read_sas('comp.sas7bdat', encoding='latin-1') #fp3v2
FF4 = pd.read_sas('ff4data.sas7bdat', encoding='latin-1')
COMP_ANNUAL = pd.read_sas('comp_annual.sas7bdat', encoding='latin-1') # fp3v3
INDUSTRY =  pd.read_csv('industries.csv')

In [11]:
def fill_zero(df, var):
    temp = df[var].isna().sum()
    df[var].fillna(0, inplace=True)
    print("NAN values for " + str(var) + ": " + str(temp))
    temp = df[var].isna().sum()
    print("NAN values for " + str(var) + " filled: " + str(temp) + " remaining")

def calculate_credit_metrics(compustat_data):
    credit_metrics_df = pd.DataFrame()

    # Rename 
    credit_metrics_df["Total_Assets"] = compustat_data["AT"]
    credit_metrics_df["Current_Liabilities"] = compustat_data["LCO"]
    credit_metrics_df["Long_Term_Debt"] = compustat_data["DLTT"]
    credit_metrics_df["Total_Liabilities"] = compustat_data["LT"]
    credit_metrics_df["Preferred_Stock"] = compustat_data["PSTK"]
    credit_metrics_df["Interest_Expense"] = compustat_data["XINT"]
    credit_metrics_df["Cash_Short_Term_Investments"] = compustat_data["CHE"]
    credit_metrics_df["Net_Income"] = compustat_data["NI"]
    credit_metrics_df["Inventory"] = compustat_data["INVT"]
    credit_metrics_df["Total_Equity"] = compustat_data["CEQ"]

    # Calculate financial ratios
    credit_metrics_df["Debt_Equity_Ratio"] = credit_metrics_df["Total_Liabilities"] / credit_metrics_df["Total_Equity"]
    credit_metrics_df["Debt_Ratio"] = credit_metrics_df["Total_Liabilities"] / credit_metrics_df["Total_Assets"]
    credit_metrics_df["Current_Ratio"] = credit_metrics_df["Current_Liabilities"] / credit_metrics_df["Total_Assets"]
    credit_metrics_df["Quick_Ratio"] = (credit_metrics_df["Current_Liabilities"] - credit_metrics_df["Inventory"]) / credit_metrics_df["Total_Assets"]
    credit_metrics_df["Interest_Coverage_Ratio"] = credit_metrics_df["Net_Income"] / credit_metrics_df["Interest_Expense"]
    credit_metrics_df["Return_On_Assets"] = credit_metrics_df["Net_Income"] / credit_metrics_df["Total_Assets"]
    credit_metrics_df["Return_On_Equity"] = credit_metrics_df["Net_Income"] / credit_metrics_df["Total_Equity"]
    credit_metrics_df["Asset_Turnover_Ratio"] = credit_metrics_df["Net_Income"] / credit_metrics_df["Total_Assets"]
    credit_metrics_df["Inventory_Turnover_Ratio"] = credit_metrics_df["Net_Income"] / credit_metrics_df["Inventory"]

    return credit_metrics_df

def calculate_credit_score(credit_metrics_df):
    # Define weights for each financial metric
    weights = {
        "Debt_Equity_Ratio": 0.2,
        "Debt_Ratio": 0.1,
        "Current_Ratio": 0.1,
        "Quick_Ratio": 0.1,
        "Interest_Coverage_Ratio": 0.1,
        "Return_On_Assets": 0.1,
        "Return_On_Equity": 0.1,
        "Asset_Turnover_Ratio": 0.1,
        "Inventory_Turnover_Ratio": 0.1
    }

    # Calculate the weighted sum of normalized metrics
    credit_score = (credit_metrics_df * pd.Series(weights)).sum(axis=1)

    return credit_score

In [12]:
class Factors:
  @staticmethod
  def _create_hedge_portfolio(data: pd.DataFrame, factor_col: str, q=10, direction=1) -> pd.DataFrame:
    """
    Creates a hedge portfolio, using D10 - D1 returns by default.

    Args:
      data (pd.DataFrame): Data, must contain columns ['monthid', 'RET', factor_col]
      factor_col (str): Name of factor column, i.e. ep1
      q (int, optional): Defaults to deciles (10).
      direction (int, optional): Either 1 or -1. If 1, we do High - Low. Otherwise, Low - High.

    Returns: Return of hedge portfolio at each monthid (i.e. the factor)
    """
    factor = []
    index = []  # monthid - 1
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp[factor_col].transform(lambda x: pd.qcut(x.rank(method="first"), 10, labels=False) if not np.isnan(x).all() else x)  # if statement in case all NaN
    for monthid, mdata in mth_grp:
      # TODO: can add value weighting of returns here if anyone needs it
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == q-1]['RET'].mean()
      factor.append(direction * (d10 - d1))
      index.append(monthid - 1)
    return pd.DataFrame(factor, index=index)
  
  @staticmethod
  def dNoa(data):
    data = data.copy()

    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)
    
    # annual_data = COMP_ANNUAL
    fill_zero(annual_data, 'DLC')
    fill_zero(annual_data, 'DLTT')
    fill_zero(annual_data, 'MIB')
    fill_zero(annual_data, 'PSTK')

    operating_assets = annual_data["AT"] - annual_data["CHE"]
    operating_liabilities = annual_data["AT"] - annual_data["DLC"] - annual_data["DLTT"] - annual_data["MIB"] - annual_data["PSTK"] - annual_data["CEQ"]

    Noa = operating_assets - operating_liabilities

    lagged_total_assets = annual_data["AT"].shift(1)  # 1-year-lagged total assets
    dNoa = (Noa - Noa.shift(1)) / lagged_total_assets

    annual_data["dNoa"] = dNoa
    annual_data["dNoa"].fillna(0, inplace=True)

    data = sqldf("SELECT a.*, b.dNoa \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['dNoa'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor

  @staticmethod
  def Nsi(data):
    data = data.copy()

    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)

    annual_data["CSHO_t_minus_1_adjusted"] = annual_data["CSHO"] * annual_data["AJEX"]

    test_shifted = annual_data.shift(periods=1)
    annual_data["CSHO_t_minus_2_adjusted"] = test_shifted["CSHO"] * test_shifted["AJEX"]

    annual_data["Nsi_u"] = np.log(annual_data["CSHO_t_minus_1_adjusted"] / annual_data["CSHO_t_minus_2_adjusted"])
    annual_data["Nsi_u"].fillna(0, inplace=True)

    annual_data["Nsi"] = np.where(annual_data["Nsi_u"] < 0, np.where(annual_data["Nsi_u"] < annual_data["Nsi_u"].quantile(0.5), 1, 2),
                             np.where(annual_data["Nsi_u"] == 0, 3,
                                      np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.1), 4,
                                               np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.2), 5,
                                                        np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.3), 6,
                                                                 np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.4), 7,
                                                                          np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.5), 8,
                                                                                   np.where(annual_data["Nsi_u"] <= annual_data["Nsi_u"].quantile(0.6), 9, 10))))))))
    annual_data["Nsi"]

    data = sqldf("SELECT a.*, b.Nsi \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['Nsi'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor

  @staticmethod
  def dNca(data):
    data = data.copy()

    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)
    
    fill_zero(annual_data, "IVAO")

    annual_data["Nca"] = annual_data["AT"] - annual_data["ACT"] - annual_data["IVAO"]
    annual_data["dNca_o"] = annual_data["Nca"].diff()
    annual_data["dNca_o"].fillna(0, inplace=True)

    total_assets_t_minus_2 = annual_data['AT'].iloc[-3]

    annual_data["dNca"] = annual_data["dNca_o"] / total_assets_t_minus_2

    data = sqldf("SELECT a.*, b.dNca \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['dNca'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor

  @staticmethod
  def dFnl(data):
    data = data.copy()

    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)
    
    fill_zero(annual_data, "DLTT")
    fill_zero(annual_data, "DLC")
    fill_zero(annual_data, "PSTK")

    annual_data["Fnl"] = annual_data["DLTT"] + annual_data["DLC"] + annual_data["PSTK"]
    annual_data["dFnl_o"] = annual_data["Fnl"].diff()
    annual_data["dFnl_o"].fillna(0, inplace=True)
    total_assets_t_minus_2 = annual_data['AT'].iloc[-3]

    annual_data["dFnl"] = annual_data["dFnl_o"] / total_assets_t_minus_2

    data = sqldf("SELECT a.*, b.dFnl \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['dFnl'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor

  @staticmethod
  def creditrisk(data):
    data = data.copy()

    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)
    
    credit_metrics = calculate_credit_metrics(annual_data)
    credit_score = calculate_credit_score(credit_metrics)
    annual_data["cdrk"] = credit_score

    data = sqldf("SELECT a.*, b.cdrk \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['cdrk'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor


  @staticmethod
  def epq1(data):
    factor = []
    data = data.copy()
    data = data[data['IBQ'] >= 0]
    data['Epq1'] = data['IBQ'] * 1000000 / data['MKTCAP']
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['Epq1'].transform(lambda x: pd.qcut(x, 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d1 - d10)
    return factor
  
  @staticmethod
  def ioca(data):
    CPI = pd.read_csv('cpi.csv')
    g = .10
    d = .15
    factor = []
    annual_data = sqldf("SELECT a.*, b.* \
                            FROM COMP_ANNUAL as a \
                            INNER JOIN INDUSTRY as b \
                            ON a.SIC >= b.lhs and a.SIC <= b.rhs")
    annual_data['year'] = annual_data['FYEAR'].astype(int)
    annual_data['LPERMNO'] = annual_data['LPERMNO'].astype(int)
    annual_data['XSGA'].fillna(0, inplace=True)
    annual_data = pd.merge(annual_data, CPI, on='year')
    annual_data.sort_values(by=['LPERMNO', 'year', 'XSGA'], inplace=True)
    annual_data.drop_duplicates(subset=['LPERMNO', 'year'], keep='last', inplace=True)
    grouped = annual_data.groupby('LPERMNO')
    oc = []
    for _, grp in grouped:
      prev = None
      for _, row in grp.iterrows():
        if prev == None:
          prev = row['XSGA'] / (g+d)
        else:
          prev = (1 - d) * prev + row['XSGA'] / row['CPI']
        oc.append(prev)
    annual_data['oc'] = oc
    annual_data = annual_data[(annual_data['oc'] > 0) & (annual_data['AT'] > 0)]
    annual_data['oca'] = annual_data['oc'] / annual_data['AT']
    def yearly_winsorize(srs):
      p1, p99 = np.nanpercentile(srs, [1, 99])
      return np.clip(srs, p1, p99)
    annual_data['ioca'] = annual_data.groupby('year')['oca'].transform(yearly_winsorize)
    annual_data['ioca'] = annual_data.groupby('industry')['ioca'].transform(lambda x: (x - x.mean()) / x.std())
    data = sqldf("SELECT a.*, b.ioca \
                  FROM data as a \
                  INNER JOIN annual_data as b \
                  ON a.PERMNO = b.LPERMNO and \
                  ((a.year = b.year and a.month >= 7) or \
                    (a.year = b.year + 1 and a.month <= 6))")
    mth_grp = data.groupby('monthid')
    data['rank'] = mth_grp['ioca'].transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))
    for _, mdata in mth_grp:
      d1 = mdata[mdata['rank'] == 0]['RET'].mean()
      d10 = mdata[mdata['rank'] == 9]['RET'].mean()
      factor.append(d10 - d1)
    return factor
  
  @staticmethod
  def ra26(data):
    factor = []
    data = data.sort_values(['PERMNO','monthid'])
    datagrp = data.groupby('PERMNO')
    for _, grp in datagrp:
      for mid in grp['monthid']:
        tm = grp[grp['monthid'].isin([mid - 24, mid - 36, mid - 48, mid - 60])]
        if (len(tm) < 4):
          factor.append(np.NaN)
        else:
          factor.append(tm['RET'].mean())
    data['ra26'] = factor
    return Factors._create_hedge_portfolio(data, 'ra26')

  @staticmethod
  def Abr1(data):
    factor = []
    data = data.copy()

    # RET_DAILY = pd.read_sas('ret_daily.sas7bdat', encoding='latin-1')  sas is SLOW
    ret_daily = pd.read_parquet("ret_daily.parquet")
    vwret = pd.read_sas('us_vwret.sas7bdat', encoding='latin-1').set_index("date")

    ret_daily["monthid"] = (ret_daily.DATE.dt.year-1975)*12 + ret_daily.DATE.dt.month

    stock_returns = ret_daily.set_index(["PERMNO", "DATE"]).sort_index()
    del ret_daily  # save memory
    # faster to pivot first and then do the operations, rather than doing a groupby
    stock_returns = stock_returns.pivot_table(values="RET", columns="PERMNO", index="DATE")
    stock_returns = stock_returns.sub(vwret.VWRETD, axis=0)  # return over market

    # get all 4-period returns from T-3 to T+1
    stock_returns = stock_returns.rolling(4).sum().shift(-1)
    stock_returns = pd.DataFrame(stock_returns.stack())
    stock_returns = stock_returns.rename(columns={0:"Abr"})

    # Merge on RDQ date so we only have the returns around earnings date
    rdqs = data[["PERMNO", "RDQ"]].dropna()
    rdqs.RDQ = pd.to_datetime(rdqs.RDQ)
    rdqs = rdqs.drop_duplicates()
    rdqs = rdqs.merge(stock_returns, left_on=["RDQ", "PERMNO"], right_index=True, how="left")
    del stock_returns  # save memory
    # This is the date we use to calculate the monthid to avoid lookahead bias,
    # since Abr is calculated using RDQ-3 to RDQ+1 returns.
    rdqs["RDQ+1"] = rdqs.RDQ + pd.offsets.BDay(1)
    rdqs["monthid"] = (rdqs["RDQ+1"].dt.year-1975)*12 + rdqs["RDQ+1"].dt.month

    rdqs = rdqs.set_index(["PERMNO", "monthid"]).sort_index()

    # duplicates can occur where RDQ is in the same month. We drop the first month-PERMNO duplicate
    rdqs = rdqs[~rdqs.index.duplicated(keep="last")]

    final = pd.DataFrame(index=data.set_index(["PERMNO", "monthid"]).index).sort_index()
    final["Abr1"] = rdqs.Abr
    # forward fill only 6 months to avoid stale data
    # TODO: could possibly be problems if data skips some monthids, but we still have no lookahead bias, just possible stale data
    final = final.groupby(level=0).ffill(limit=6)

    data = data.merge(final, how="left", left_on=["PERMNO", "monthid"], right_index=True).sort_values(by=['PERMNO', 'monthid'])

    return Factors._create_hedge_portfolio(data, "Abr1", q=10, direction=1)
    # return data

  @staticmethod
  def e11(data):
    """
    Calculate ε11 factor (Residual momentum, prior 11-month returns) with 1 month holding period.
    """
    data = data.copy()
    
    # Get a series of rf indexed by date
    rf = data.set_index("monthid")[["RF"]].reset_index().drop_duplicates(subset="monthid")
    rf = rf.set_index("monthid").sort_index().RF

    # Get xret with each permno in a column
    xret = data.loc[:,~data.columns.duplicated()].pivot_table(index="monthid", columns="PERMNO", values="RET").sort_index()
    xret = xret.sub(rf, axis=0)

    # Get ff data with constant
    ff_3 = data[["monthid", "SMB", "HML", "MKTRF"]].drop_duplicates(subset="monthid").set_index("monthid").sort_index()
    ff_3 = sm.add_constant(ff_3)

    from numpy.linalg import pinv

    def last_ff_residual(series: pd.Series, ff: pd.DataFrame) -> float:
      """Computes FF residuals for a series of excess returns.

      Args:
        series (pd.DataFrame): rolling excess returns. Must have no null values.
        ff (pd.DataFrame): factors, must have overlapping index with `series`. All columns are used as factors. Must have constant column added.

      Returns: residual on last date T
      """
      # y = series
      x = ff.loc[series.index]
      
      # ffmodel = sm.OLS(y, x).fit()
      # residual_values = ffmodel.resid

      # sm.OLS is too slow, do it with linear algebra instead
      params = pinv(x).dot(series)

      # calculate last residual
      t_residual = series.iloc[-1] - x.iloc[-1].dot(params)
      
      return t_residual

    residuals = xret.rolling(window=36, min_periods=36).apply(
      lambda series: last_ff_residual(series, ff_3)
    )
    scaled_residuals = residuals / residuals.rolling(36, min_periods=12).std()

    # The time T residual momentum is the sum of residual returns for T-12, T-11, ... T-1
    # We do this with a rolling 11 period sum, and then shift down by 1
    e11 = scaled_residuals.rolling(11).sum().shift()

    # Reshape so the columns are [monthid, PERMNO, e11] and merge into data
    e11 = pd.DataFrame(e11.stack()).rename(columns={0: "e11"})
    data = data.merge(e11, how="left", left_on=["monthid", "PERMNO"], right_index=True)

    return Factors._create_hedge_portfolio(data, "e11", q=10, direction=1)

  @staticmethod
  def Re1(data):
    data = data.copy()

    ibes_link = pd.read_csv("crsp_ibes_link.csv")  # ibes ticker to PERMNO map  https://wrds-www.wharton.upenn.edu/pages/get-data/linking-suite-wrds/ibes-crsp-link/
    ibes_eps = pd.read_csv("ibes_eps_estimate.csv")  # Mean EPS estimates  https://wrds-www.wharton.upenn.edu/pages/get-data/ibes-thomson-reuters/ibes-academic/summary-history/summary-statistics/

    ibes_link = ibes_link.dropna()

    # Only want USD currency
    ibes_eps = ibes_eps[ibes_eps.CURCODE == "USD"]

    # Get the estimates with the date and permno
    eps_est = sqldf(
      """
      select l.permno, e.STATPERS as DATE, e.MEANEST
      from ibes_eps e
      left join ibes_link l
      on e.ticker = l.TICKER and e.STATPERS >= l.sdate and e.STATPERS <= l.edate 
      """
    )

    # Add monthid
    eps_est = eps_est.drop_duplicates(subset=["DATE", "PERMNO"])
    eps_est.DATE = pd.to_datetime(eps_est.DATE)
    eps_est['monthid'] = (eps_est.DATE.dt.year-1975)*12 + eps_est.DATE.dt.month

    # Merge close price
    eps_est = eps_est.merge(data.set_index(["monthid", "PERMNO"]).PRC, on=["monthid", "PERMNO"])
    eps_est = eps_est.set_index(["PERMNO", "monthid"]).sort_index()

    # Calculate factor value for each stock

    # TODO: we should make PRC the split adjusted price as our EPS values are split adjusted
    re = eps_est.reset_index().set_index("monthid").sort_index().groupby(["PERMNO"]).apply(
        # 6 period rolling sum of (f_t - f_t-1) / p_t-1, requiring at least 4 consecutive observations
        lambda df: ((df.MEANEST - df.MEANEST.shift()) / df.PRC.shift()).rolling(window=6, min_periods=4).sum()
    )
    eps_est["Re1"] = re
    
    data = data.merge(eps_est[["Re1"]], how="left", on=["monthid", "PERMNO"])
    
    return Factors._create_hedge_portfolio(data, "Re1", q=10, direction=1)
  

  # D. Investment ==========================================
  @staticmethod 
  def _compute_shifted(data, factor_col, input_col1, input_col2, shift_amt, op='-'):
    """
    for factor calculations that require a shift of data values to avoid
    lookahead bias:
    Input:
      data: DataFrame
      factor_col: name of factor to be computed
      input_col: name of column used in the factor computation
      shift_amt: number of months to lag by for input_col
    Return:
      a dataframe with cols ['monthid', 'PERMNO', 'factor'] for merge
      into the overall data
    """
    permno_grp = data.groupby('PERMNO')
    # construct a mini dataframe of shifted values -> (monthid, permno, dRoa1 computed for the relevant month)
    factor_values = {'monthid': [], 'PERMNO': [], factor_col:[]}

    for _, pdata in permno_grp:
      # sort by monthid
      pdata = pdata.sort_values('monthid') 
      # lag Roa to avoid lookahead bias
      if (op == '-'):
        pdata[factor_col] = pdata[input_col1] - pdata[input_col2].shift(shift_amt)
      else:
        pdata[factor_col] = pdata[input_col1] / pdata[input_col2].shift(shift_amt)
      # add to dataframe
      
      factor_values['PERMNO'].extend(pdata['PERMNO'].tolist())
      factor_values['monthid'].extend(pdata['monthid'].tolist())
      factor_values[factor_col].extend(pdata[factor_col].tolist())

    factor_values = pd.DataFrame(factor_values)
    factor_values['monthid'] = factor_values['monthid'].astype(np.int64)
    factor_values['PERMNO'] = factor_values['PERMNO'].astype(np.int64)

    return pd.DataFrame(factor_values)
    
  @staticmethod
  def droe1(data):
    """dRoe: return on equity minus its value from four quarters ago """
    #   for each month t, sort all stocks into deciles based on their most recent past dRoe
    #   monthly decile returns calculated for the current month t

    # ROE = NET (RDQ) / Equity (CEQQ)   
    data = data.copy()
    data['Roe'] = data['NIQ'] / data['CEQQ']

    # require that earnings announcement date is after the current fiscal quarter end
    data = data[~(data['DATADATE'] > data['RDQ'])]

    # compute factor values while accounting for lagged inputs
    factor_values = Factors._compute_shifted(data, factor_col='dRoe1', input_col1='Roe', input_col2='Roe', shift_amt=12, op='-')
    # merge factor values back into the overall data frame
    data = data.merge(factor_values, on=["monthid", "PERMNO"])
    # then construct decile portfolios
    return Factors._create_hedge_portfolio(data, "dRoe1", q=10)
  
  @staticmethod
  def droa1(data):
    """ droa1 "is return on assets minus its value from four quarters ago" """
    factor = []
    data = data.copy()
    data['Roa'] = data['NIQ'] / data['ATQ']

    # require that earnings announcement date is after the current fiscal quarter end
    data = data[~(data['DATADATE'] > data['RDQ'])]
    # compute factor values while accounting for lagged inputs
    factor_values = Factors._compute_shifted(data, factor_col='dRoa1', input_col1='Roa', input_col2='Roa', shift_amt=12, op='-')
    # merge factor values back into the overall data frame
    data = data.merge(factor_values, on=["monthid", "PERMNO"])
    # then construct decile portfolios
    return Factors._create_hedge_portfolio(data, "dRoa1", q=10)

  @staticmethod
  def rnaq1(data):
    """ Rnaq1: Quarterly return on net operating assets """

    data = data.copy()
    # zero out NAs as specified in the paper
    fill_na_cols = ['IVAOQ', 'DLCQ', 'DLTTQ', 'MIBQ', 'PSTKQ'] 
    data[fill_na_cols] = data[fill_na_cols].fillna(0)
    # operating assets = ATQ - CHEQ - IVAOQ
    data['oa'] = data['ATQ'] - data['CHEQ'] - data['IVAOQ']
    # operating liabilities = ATQ - DLCQ - DLTTQ - MIBQ - PSTKQ - CEQQ
    data['ol'] = data['ATQ'] - data['DLCQ'] - data['DLTTQ'] - data['MIBQ'] - data['PSTKQ'] - data['CEQQ']
    # net operating assets = OA - OL
    data['noa'] = data['oa'] - data['ol']

    # compute factor values while accounting for lagged inputs - 1 quarter lagged noa
    factor_values = Factors._compute_shifted(data, factor_col='rnaq1', input_col1='OIADPQ', input_col2='noa', shift_amt=4, op='/')
    # merge factor values back into the overall data frame
    data = data.merge(factor_values, on=["monthid", "PERMNO"])

    return Factors._create_hedge_portfolio(data, 'rnaq1')

  @staticmethod
  def atoq1(data):
    """ atoq1: quarterly sales divided by 1-quarter-lagged Noa """
    data = data.copy()
    # zero out NAs as specified in the paper
    fill_na_cols = ['IVAOQ', 'DLCQ', 'DLTTQ', 'MIBQ', 'PSTKQ'] 
    data[fill_na_cols] = data[fill_na_cols].fillna(0)
    # operating assets = ATQ - CHEQ - IVAOQ
    data['oa'] = data['ATQ'] - data['CHEQ'] - data['IVAOQ']
    # operating liabilities = ATQ - DLCQ - DLTTQ - MIBQ - PSTKQ - CEQQ
    data['ol'] = data['ATQ'] - data['DLCQ'] - data['DLTTQ'] - data['MIBQ'] - data['PSTKQ'] - data['CEQQ']
    # net operating assets = OA - OL
    data['noa'] = data['oa'] - data['ol']

    # compute factor values while accounting for lagged inputs - 1 quarter lagged noa
    factor_values = Factors._compute_shifted(data, factor_col='atoq1', input_col1='SALEQ', input_col2='noa', shift_amt=4, op='/')
    # merge factor values back into the overall data frame
    data = data.merge(factor_values, on=["monthid", "PERMNO"])

    return Factors._create_hedge_portfolio(data, 'atoq1')
    

In [19]:
# add RET_T1 column to data
returns = assets.data[["RET", "monthid", "PERMNO"]].pivot_table(values="RET", index="monthid", columns="PERMNO")
fwd_ret = returns.shift(-1).stack()
fwd_ret.name = "RET_T1"
test = assets.data.merge(fwd_ret, how="left", left_on=["monthid", "PERMNO"], right_index=True)

In [21]:
test[test.PERMNO == 10025]

,PERMNO,DATE,SHRCD,EXCHCD,PRC,RET,SHROUT,year,month,monthid,...,PSTKQ,SALEQ,qtrid,DATEFF,SMB,HML,MKTRF,RF,UMD,RET_T1
0,10025,2015-05-29 00:00:00.000000,11.0,3.0,50.029999,-0.001397,5103.0,2015,5,485,...,0.0,275.439,481,2015-05-29,0.0093,-0.0114,0.0136,0.0000,0.0582,0.103338
203,10025,2015-06-30 00:00:00.000000,11.0,3.0,55.200001,0.103338,5103.0,2015,6,486,...,0.0,275.439,481,2015-06-30,0.0291,-0.0079,-0.0153,0.0000,0.0301,-0.124094
494,10025,2015-07-31 00:00:00.000000,11.0,3.0,48.349998,-0.124094,5103.0,2015,7,487,...,0.0,275.439,481,2015-07-31,-0.0417,-0.0413,0.0154,0.0000,0.0998,0.126163
3357,10025,2015-08-31 00:00:00.000000,11.0,3.0,54.450001,0.126163,5103.0,2015,8,488,...,0.0,285.722,484,2015-08-31,0.0033,0.0277,-0.0604,0.0000,-0.0209,0.052893
6197,10025,2015-09-30 00:00:00.000000,11.0,3.0,57.330002,0.052893,5103.0,2015,9,489,...,0.0,285.722,484,2015-09-30,-0.0262,0.0056,-0.0307,0.0000,0.0522,0.395430
8989,10025,2015-10-30 00:00:00.000000,11.0,3.0,80.000000,0.395430,5103.0,2015,10,490,...,0.0,285.722,484,2015-10-30,-0.0188,-0.0046,0.0775,0.0000,-0.0328,0.139750
11775,10025,2015-11-30 00:00:00.000000,11.0,3.0,91.180000,0.139750,5103.0,2015,11,491,...,0.0,301.982,487,2015-11-30,0.0360,-0.0042,0.0056,0.0000,0.0228,-0.153871
14544,10025,2015-12-31 00:00:00.000000,11.0,3.0,77.150002,-0.153871,5103.0,2015,12,492,...,0.0,301.982,487,2015-12-31,-0.0284,-0.0261,-0.0217,0.0001,0.0336,0.100324
17279,10025,2016-01-29 00:00:00.000000,11.0,3.0,84.639999,0.100324,5107.0,2016,1,493,...,0.0,301.982,487,2016-01-29,-0.0342,0.0209,-0.0577,0.0001,0.0144,-0.070061
20067,10025,2016-02-29 00:00:00.000000,11.0,3.0,78.709999,-0.070061,5107.0,2016,2,494,...,0.0,278.248,490,2016-02-29,0.0073,-0.0057,-0.0007,0.0002,-0.0438,-0.161479


In [13]:
class Assets:
  crsp = CRSP.copy()
  comp = COMP.copy()
  ff4 = FF4.copy()
  fact = pd.DataFrame()
  data = pd.DataFrame()
  train_start = '1975-01-01'
  train_end = '2005-12-31'
  # test_start = '2006-01-01'
  # test_end = '2020-12-31'
  factors = {
    'epq1': Factors.epq1,
    # 'ioca': Factors.ioca,
    # 'ra26': Factors.ra26,
    # 'e11': Factors.e11,  # comment out for now as this takes ~12min to run
    # 'Re1': Factors.Re1,
    # 'Abr1': Factors.Abr1,

    # D. Investment =====================================
    # 'dRoe1': Factors.droe1,
    'dRoa1': Factors.droa1,
    # 'rnaq1': Factors.rnaq1,
    # 'atoq1': Factors.atoq1,

    # 'dNoa': Factors.dNoa,
    # 'Nsi': Factors.Nsi,
    # 'dNca': Factors.dNca,
    # 'dFnl' : Factors.dFnl,
    # 'creditrisk': Factors.creditrisk
  }
  factor_fm = pd.DataFrame(columns=['mean','std_error','tstat'])
  
  def __init__(self, start_date=None):

    # Make testing faster by optiSonally limiting dates
    if start_date is not None:
      self.crsp = self.crsp[self.crsp.DATE >= start_date]
      self.comp = self.comp[self.comp.DATADATE >= start_date]

    self.clean_crsp()
    self.clean_comp()
    self.clean_ff4()
    self.illiquidity_filter()
    
    self.merge_data()
    self.gen_factors()
    self.fama_macbeth()
    
  def clean_crsp(self):
    self.crsp['PERMNO'] = self.crsp['PERMNO'].astype(int)
    self.crsp['year'] = self.crsp['DATE'].dt.year
    self.crsp['month'] = self.crsp['DATE'].dt.month
    self.crsp['monthid'] = (self.crsp['year']-1975)*12 + self.crsp['month']
    self.crsp['PRC'] = self.crsp['PRC'].apply(lambda x: x if x > 0 else x * -1)
    
  def clean_comp(self):
    self.comp['qtrid'] = (self.comp['DATADATE'].dt.year-1975)*12 + self.comp['DATADATE'].dt.month
    self.comp['LPERMNO'] = self.comp['LPERMNO'].astype(int)
    self.comp.drop(columns=['CONSOL', 'INDFMT', 'DATAFMT', 'POPSRC', 'DATAFQTR', 'DATACQTR', 'CURCDQ', 'COSTAT'], 
                  inplace=True)
    
  def clean_ff4(self):
    self.ff4['monthid'] = (self.ff4['DATEFF'].dt.year-1975)*12 + self.ff4['DATEFF'].dt.month
    self.fact = self.ff4.copy()
    
  def illiquidity_filter(self):
    self.crsp = self.crsp[self.crsp['PRC'] >= 5]
    self.crsp['MKTCAP'] = self.crsp['PRC'] * self.crsp['SHROUT'] * 1000
    tmp = {}
    grp = self.crsp[(self.crsp['month'] == 1) & (self.crsp['MKTCAP'] >= 100000000)].groupby('year')
    for yr, group in grp:
      tmp[yr] = list(group['PERMNO'])
    liquidity = self.crsp.groupby(['year'])['PERMNO'].transform(lambda x: x.isin(tmp[x.name]))
    self.crsp = self.crsp[liquidity]
    
  def merge_data(self):
    lhs = self.crsp
    rhs = self.comp
    self.data = sqldf("SELECT a.*, b.* \
                       FROM lhs as a \
                       INNER JOIN rhs as b \
                       ON a.PERMNO = b.LPERMNO and a.monthid >= b.qtrid + 4 and a.monthid <= b.qtrid + 6")
    self.data.drop_duplicates(subset=['PERMNO', 'monthid'], keep='last', inplace=True)
    self.data = self.data.loc[:,~self.data.columns.duplicated()].copy()  # remove duplicate col
    self.data = pd.merge(self.data, self.ff4, on='monthid')
  
  def gen_factors(self):
    for factor, func in self.factors.items():
      try:
        self.fact[factor] = func(self.data)
      except Exception as e:
        print(f"Error generating {factor=}: {e}")
        print(f"SKIPPING THIS FACTOR")
    self.fact = pd.merge(self.fact, self.data[['monthid','RET','PERMNO']], on='monthid')
  
  def fama_macbeth(self):
    for factor, func in self.factors.items():
      try:
        fmb = self.fact[['DATEFF','monthid','RET',factor,'SMB','HML','MKTRF','UMD']].copy()
        fmb = fmb[(fmb['DATEFF'] >= self.train_start) & (fmb['DATEFF'] <= self.train_end)]
        fmb.dropna(inplace=True)
        result = fama_macbeth(fmb,'monthid','RET',[factor,'SMB','HML','MKTRF','UMD'],intercept=True)
        self.factor_fm.loc[factor] = fm_summary(result).loc[factor]
      except Exception as e:
        print(f"Error running Fama Macbeth on {factor=}: {e}")

  def gen_factor_betas(self, train_start='1975-01-01', train_end='2005-12-31'):
    """ 
      testing sample forecasting for linear models
    """
    # initialize a dictionary to store factor betas, where beta columns
    # are named {factor}_b
    factor_betas = {'PERMNO':[], 'monthid': [], 'alpha':[]}
    factor_cols = []
    factor_list = list(self.factors.keys())
    for factor in self.factors.keys():
      factor_b = factor + "_b"
      # factor_list.append(factor)
      factor_cols.append(factor_b)
      factor_betas[factor_b] = []

    # get coefficients for factors through regression
    # combine by stock first, then regress across times
    # TODO rolling 36 months
    data = self.fact.copy()

    # filter data period
    data = data[(data['DATEFF'] >= train_start) & (data['DATEFF'] <= train_end)]
    permno_grp = data.groupby('PERMNO')

    for permno, pdata in permno_grp:
      # dropna, TODO consider alternative
      pdata = pdata.dropna()
      # check number of observations available
      if (len(pdata.index) < 12):
        continue
      # sort by monthid
      pdata = pdata.sort_values('monthid') 
      independents = pdata[factor_list]
      # since factors were constructed accounting for lookahead bias (via lags 
      # when necessary), this step regresses factor values at time t to returns 
      # at time t
      model = linear_model.LinearRegression().fit(independents, pdata["RET"])
      # add values to factor coefficients dataframe
      factor_betas['PERMNO'].append(pdata['PERMNO'].iloc[0])
      factor_betas['monthid'].append(pdata['monthid'].iloc[0])
      factor_betas['alpha'].append(model.intercept_)
      for i, factor_col in enumerate(factor_cols): 
        factor_betas[factor_col].append(model.coef_[i])

    factor_betas_df = pd.DataFrame(factor_betas)
    factor_betas_df.sort_values(["PERMNO", "monthid"])

    return factor_betas_df
    
  
  def score_linear(self):
    """ 
      score stocks during the out of sample period
    """
    # get factor coefficients from regression
    data = self.data.copy()
    stock_data = self.data.copy()
    factor_betas = self.gen_factor_betas()
    factor_list = list(self.factors.keys())
    data = pd.merge(data, self.fact, on=["PERMNO", "monthid"])
    data = pd.merge(data, factor_betas, on=["PERMNO", "monthid"])

    # add RET_T1 column to data
    returns = data[["RET", "monthid", "PERMNO"]].pivot_table(values="RET", index="monthid", columns="PERMNO")
    fwd_ret = returns.shift(-1).stack()
    fwd_ret.name = "RET_T1"
    data = data.merge(how="left", left_on=["monthid", "PERMNO"], right_index=True)

    # determine holdings based on ranks - for each time point, have a 
    # list of permnos for top and lowest deciles 
    # compute returns based on the given formula
    data['pre_ret'] = data['alpha']
    data['pre_ret'] = data['pre_ret'].fillna(0)
    for factor in factor_list:
      factor_b = factor+"_b"
      data['pre_ret'] = data['pre_ret'] + (data[factor] * data[factor_b])
    
    # rank stocks for each out-of-sample period and thus rebalance at each time
    # we want to use the previous month to predict the next, thus use predicted return
    # at t-1 for time t's portfolio construction
    grouped_data = []
    permno_group = data.groupby('PERMNO')
    for _, pdata in permno_group:
      # sort by monthid, then shift to get t-1 return
      pdata = pdata.sort_values(["monthid"])
      pdata['pre_ret_t1'] = pdata['pre_ret'] # TODO issue here
      # pdata['pre_ret_t1'] = pdata['pre_ret'].shift(1)
      grouped_data.append(pdata)
    # merge back into overall data 
    data = pd.concat(grouped_data)

    # store what permnos to construct the long / short portfolio with 
    # initialize a dataframe to track decile 10 and 1 portfolios
    # rank stocks to deciles based on the values calculated from the predication equation
    data['rank'] = data['pre_ret_t1'].transform(lambda x: pd.qcut(x.rank(method="first"), 10, labels=False) if not np.isnan(x).all() else x)  # if statement in case all NaN
    decile_permnos = {'monthid': [], 'd1_permnos':[],  'd10_permnos': []}
    
    mth_group = data.groupby('monthid')
    for month, mdata in mth_group:
      # skipping ones with very little data
      if (len(mdata['pre_ret_t1']) < 12):
        continue
      # construct equal weighted decile portfolios
      decile_permnos['monthid'].append(mdata['monthid'].iloc[0])
      decile_permnos['d1_permnos'].append(mdata[mdata['rank']==0]['PERMNO'])
      decile_permnos['d10_permnos'].append(mdata[mdata['rank']==9]['PERMNO'])

    decile_permnos_df = pd.DataFrame(decile_permnos)
    decile_permnos_df.sort_values(['monthid'])

    # for each time point, we have the permnos of the 1st and 10th decile 
    # construct equal weighted portfolios 
    decile_rets = {'monthid': [], 'd1_ret': [], 'd10_ret': []}
    mth_groups = stock_data.groupby('monthid')
    for _, mdata in mth_groups:
      monthid = mdata['monthid'].iloc[0]
      # skipping months without data
      if not (decile_permnos_df['monthid'].isin([monthid]).any()):
        continue
      # locate the current monthid's permnos, then filter to those permno and id, and take the mean of the return column 
      d1_permnos = decile_permnos_df.loc[decile_permnos_df['monthid'] == monthid, 'd1_permnos'].values[0]
      d10_permnos = decile_permnos_df.loc[decile_permnos_df['monthid'] == monthid, 'd10_permnos'].values[0]
      # filter returns for decile 1 and decile 10 permnos
      d1_rets = mdata[mdata['PERMNO'].isin(d1_permnos)]['RET']
      d10_rets = mdata[mdata['PERMNO'].isin(d10_permnos)]['RET']
      # equal weight d1 and d10 portfolios
      decile_rets['monthid'].append(monthid)
      decile_rets['d1_ret'].append(-1 * d1_rets.mean()) # short d1 portfolio
      decile_rets['d10_ret'].append(d10_rets.mean()) 

    # return a dataframe of decile returns
    decile_rets_df = pd.DataFrame(decile_rets)
    decile_rets_df.sort_values(["monthid"], inplace=True)
    decile_rets_df['d10-d1_ret'] = decile_rets_df['d10_ret'] - decile_rets_df['d1_ret']

    capm_ret, ff_ret = Assets.capm_ff_predicted_returns(self.fact.copy())
    decile_rets_df['capm_ret'] = capm_ret
    decile_rets_df['ff_ret'] = ff_ret
    return decile_rets_df


  @staticmethod
  def capm_ff_predicted_returns(data):
    """ generate CAPM predicted and FF predicted returns"""
    # factors dataframe contains returns and factors for all permnos and monthids
    # estimate beta coefficients
    data["XRET"] = data["RET"] - data["RF"]
    data.dropna(inplace=True)

    # 2 stage regression to get predicted returns 
    permno_group = data.groupby("PERMNO")
    asset_betas = {'PERMNO': [], 'C_MKTRF_b':[], 'F_MKTRF_b': [], 'SMB_b':[], 'HML_b':[], 'UMD_b':[]}
    for permno, pdata in permno_group:
      # FF factor model to get coefficients
      ff_explanatory = pdata[['MKTRF', 'SMB', 'HML', 'UMD']] 
      ff_explanatory = sm.add_constant(ff_explanatory)
      ff_model = sm.OLS(pdata["XRET"], ff_explanatory).fit()
      ff_betas = ff_model.params
      # CAPM model
      capm_explanatory = pdata[['MKTRF']]
      capm_explanatory = sm.add_constant(capm_explanatory)
      capm_model = sm.OLS(pdata["XRET"], capm_explanatory).fit()
      capm_betas = capm_model.params
      asset_betas['PERMNO'].append(permno)
      asset_betas['C_MKTRF_b'].append(capm_betas['MKTRF'])
      asset_betas['F_MKTRF_b'].append(ff_betas['MKTRF'])
      asset_betas['SMB_b'].append(ff_betas['SMB'])
      asset_betas['HML_b'].append(ff_betas['HML'])
      asset_betas['UMD_b'].append(ff_betas['UMD'])

    asset_betas_df = pd.DataFrame(asset_betas)
    data = pd.merge(data, asset_betas_df, on=["PERMNO"])
    month_groups = data.groupby("monthid")

    print(asset_betas_df)
		# 2nd stage regression is for all assets at each time point
    risk_lambdas = {'monthid':[], 'C_MKTRF_l':[], 'F_MKTRF_l': [], 'SMB_l':[], 'HML_l':[], 'UMD_l':[]}
    for monthid, mdata in month_groups:
      ff_explanatory = mdata[['F_MKTRF_b', 'SMB_b', 'HML_b', 'UMD_b']]
      ff_explanatory = sm.add_constant(ff_explanatory)
      ff_model = sm.OLS(mdata["XRET"], ff_explanatory).fit()

      capm_explanatory = mdata[['C_MKTRF_b']]
      capm_explanatory = sm.add_constant(capm_explanatory)
      capm_model = sm.OLS(mdata["XRET"], capm_explanatory).fit()

      risk_lambdas['monthid'].append(monthid)
      risk_lambdas['C_MKTRF_l'].append(capm_model.params['C_MKTRF_b'])
      risk_lambdas['F_MKTRF_l'].append(ff_model.params['F_MKTRF_b'])
      risk_lambdas['SMB_l'].append(ff_model.params['SMB_b'])
      risk_lambdas['HML_l'].append(ff_model.params['HML_b'])
      risk_lambdas['UMD_l'].append(ff_model.params['UMD_b'])

    # with the risk lambdas, we can predict return
    risk_lambdas_df = pd.DataFrame(risk_lambdas)

    print(risk_lambdas_df)

    predict = pd.merge(data, risk_lambdas_df, on=['monthid'])
    predict['capm_ret'] = predict['RF'] + (predict['C_MKTRF_b'] * predict['C_MKTRF_l'])
    predict['ff_ret'] = predict['RF'] + (predict['SMB_b']*predict['SMB_l']) + (predict['HML_b']*predict['HML_l']) + (predict['UMD_b']*predict['UMD_l'])

    return predict['capm_ret'], predict['ff_ret']
 
  def performance_analytics(self, returns, ret_col='RET', benchmark='capm', capm_col='capm_ret', ff_col='ff_ret'):
    """ 
    takes in a dataframe of returns indexed by monthid:
    must include columns:
        * RET (for returns to calculate analytics for)
        * capm_ret (retruns generated via CAPM model)
        * ff_ret (returns generated from FF 4 model)

    compute metrics on "resulting out-of-sample returns over all months in the 
    out of sample period"
    """

    raw_ret = returns[ret_col].mean()
    capm_ret = returns[capm_col].mean()
    ff_ret = returns[ff_col].mean()

    if (benchmark=='capm'):
      benchmark_col = capm_col
      benchmark_ret = capm_ret
    else:
      benchmark_col = ff_col
      benchmark_ret = ff_ret
      
    # CAPM alpha - difference between capm predicted returns and actual returns
    capm_alpha = raw_ret - capm_ret
    # 4-Factor alpha - difference between 4 factor predicted returns and actual returns
    ff_alpha = raw_ret - ff_ret
    # sharpe ratio
    sr = (raw_ret - capm_ret) / returns[ret_col].std()
    # Information ratio 
    ir = (raw_ret - benchmark_ret) / (returns[ret_col] - returns[benchmark_col]).std()

    analytics = {'raw_return': [raw_ret] , 'sharpe_r':[sr], 'capm_alpha': [capm_alpha], 'ff_alpha': [ff_alpha], 'information_r': [ir]}
    return pd.DataFrame(analytics)



In [35]:
assets = Assets()

C:\Users\hughj\AppData\Local\Temp\ipykernel_14944\1009011452.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.crsp['MKTCAP'] = self.crsp['PRC'] * self.crsp['SHROUT'] * 1000


In [36]:
data = assets.data.copy()
stock_data = assets.data.copy()
factor_betas = assets.gen_factor_betas()

# Code to Do Fama Macbeth Predictions

In [68]:
# Hack assets to use our cache
assets.fact = pd.read_parquet("assets.parquet")

assets.factors = {
    'epq1': Factors.epq1,
    'ioca': Factors.ioca,
    'ra26': Factors.ra26,
    'e11': Factors.e11,  # comment out for now as this takes ~12min to run
    'Re1': Factors.Re1,
    'Abr1': Factors.Abr1,

    # D. Investment =====================================
    'dRoe1': Factors.droe1,
    'dRoa1': Factors.droa1,
    'rnaq1': Factors.rnaq1,
    'atoq1': Factors.atoq1,

    'dNoa': Factors.dNoa,
    'Nsi': Factors.Nsi,
    'dNca': Factors.dNca,
    'dFnl' : Factors.dFnl,
    # 'creditrisk': Factors.creditrisk
  }

In [71]:
FACTORS = list(assets.factors.keys())
FACTORS

['epq1',
 'ioca',
 'ra26',
 'e11',
 'Re1',
 'Abr1',
 'dRoe1',
 'dRoa1',
 'rnaq1',
 'atoq1',
 'dNoa',
 'Nsi',
 'dNca',
 'dFnl']

In [69]:
assets.fama_macbeth()

In [70]:
# rolling_exposures = pd.read_parquet("rolling_betas.parquet")

# Calculate rolling exposures
# copied from ml
def calculate_rolling_betas(stock_returns: pd.DataFrame, factor_returns: pd.DataFrame, lookback=36, min_periods=12):
    """Calculate rolling betas for each stock against each factor individually.

    Args:
        stock_returns (pd.DataFrame): _description_
        factor_returns (pd.DataFrame): _description_
        lookback (int, optional): _description_. Defaults to 36.
        min_periods (int, optional): _description_. Defaults to 12.

    Returns:
        _type_: _description_
    """
    assert (stock_returns.index == factor_returns.index).all()

    betas = {}

    rolling_returns = stock_returns.rolling(lookback, min_periods=min_periods)  # returns in shape (rows=monthid, cols=stock)

    for f in factor_returns:
        print(f"Computing exposures for {f}...")

        factor_return_series = factor_returns[f]

        # beta = cov(stock_return, factor_return) / factor_return.var()
        rolling_cov = rolling_returns.cov(factor_return_series)
        rolling_betas = rolling_cov.div(factor_return_series.rolling(lookback, min_periods=min_periods).var(), axis=0)
        betas[f] = rolling_betas.stack()
    
    print(f"Completed all regressions, joining results")
    all_betas = pd.concat(betas.values(), axis=1)
    all_betas.columns = betas.keys() 
    return all_betas

In [73]:
rolling_exposures = calculate_rolling_betas(returns, factors, lookback=36, min_periods=12)

# rolling_exposures = pd.read_parquet("rolling_betas.parquet")

c:\Users\hughj\anaconda3\envs\factor-modelling\lib\site-packages\fastparquet\core.py:278: UserWarning: Non-categorical multi-index is likely brittle
  warnings.warn("Non-categorical multi-index is likely brittle")
c:\Users\hughj\anaconda3\envs\factor-modelling\lib\site-packages\fastparquet\core.py:278: UserWarning: Non-categorical multi-index is likely brittle
  warnings.warn("Non-categorical multi-index is likely brittle")


In [75]:
rolling_exposures

SMB       HML     MKTRF       UMD      epq1      ioca  \
monthid PERMNO                                                               
12      10006   1.679308  0.614674  1.139007 -1.707362 -0.105393  0.109047   
        10102   1.954394  1.867570  1.670524 -2.060411 -0.304851 -0.117509   
        10137   1.898073  1.549960  1.493734 -2.289492 -0.856649 -2.235946   
        10145   0.771263  0.558086  1.094370 -0.997185  0.711606  0.955321   
        10161   1.119868  1.002363  1.177968 -1.217905 -0.582358 -0.076098   
...                  ...       ...       ...       ...       ...       ...   
552     93422   3.076077 -0.084524  1.435013 -1.510296 -0.993870  4.126767   
        93423   3.312609  1.810953  2.493631 -1.960793 -2.228965  0.443606   
        93426   2.381135  1.136426  1.476925 -0.980625 -1.141098  1.227748   
        93427   1.572988  0.532561  1.019099 -0.738630 -0.705500  0.660595   
        93436   1.022571  0.601093  2.116714 -1.160606 -0.942512 -0.111084   

                    ra26       e11      Abr1       Re1     dRoe1     dRoa1  \
monthid PERMNO                                                               
12      10006        NaN       NaN  0.610603       NaN       NaN       NaN   
        10102        NaN       NaN  1.717351       NaN       NaN       NaN   
        10137        NaN       NaN  0.759723       NaN       NaN       NaN   
        10145        NaN       NaN  0.679950       NaN       NaN       NaN   
        10161        NaN       NaN  0.767828       NaN       NaN       NaN   
...                  ...       ...       ...       ...       ...       ...   
552     93422  -2.003290 -2.055479 -0.583393 -0.109971 -2.598302 -2.637887   
        93423   1.532513 -2.624605 -0.093217 -2.253297 -5.689870 -4.353719   
        93426  -0.037411 -0.880351 -0.073059 -0.757526 -1.562708 -2.042546   
        93427   0.076590 -1.000012  0.072712 -0.681887 -1.281484 -1.208034   
        93436   0.377209 -1.394423 -0.465998 -1.886929 -5.343435 -4.543288   

                   rnaq1     atoq1      dNoa       Nsi      dNca      dFnl  
monthid PERMNO                                                              
12      10006        NaN       NaN -0.900742 -1.164136 -1.175098  0.434009  
        10102        NaN       NaN -1.053599 -0.901849 -2.382929 -3.109737  
        10137        NaN       NaN -1.680210 -3.868717 -1.879660  0.186828  
        10145        NaN       NaN -0.552862  0.737790 -1.138172 -1.040248  
        10161        NaN       NaN -0.319917 -0.510862 -0.782774 -1.419371  
...                  ...       ...       ...       ...       ...       ...  
552     93422  -1.419079 -0.332274  3.812886 -3.577442 -0.647021  0.519884  
        93423  -4.634608  0.154103  0.232444 -1.319534 -2.903477  4.069520  
        93426  -1.730868  0.300901 -0.253447 -0.390748 -1.965552  1.941938  
        93427  -0.350164  0.942320  0.285429  0.199957 -0.927963  0.927111  
        93436  -4.485997  0.707003 -1.584795 -0.099452 -3.097912  3.443065  

[1340635 rows x 18 columns]

In [78]:
factor_premia = assets.factor_fm["mean"]
factor_premia

epq1     0.001333
dRoa1   -0.000118
ioca     0.000704
ra26     0.000349
e11     -0.000442
Re1      0.000152
Abr1     0.001243
dRoe1   -0.000097
rnaq1   -0.000462
atoq1    0.000614
dNoa     0.000188
Nsi      0.001401
dNca    -0.000566
dFnl    -0.000222
Name: mean, dtype: float64

In [101]:
# Sum(Factor Premia * Factor Beta) gives us the predicted T+1 return
preds = (rolling_exposures[FACTORS] * factor_premia.loc[FACTORS]).sum(axis=1)
preds = pd.DataFrame(preds, columns=["pred"])

# Get T+1 actual returns
returns = data[["RET", "monthid", "PERMNO"]].pivot_table(values="RET", index="monthid", columns="PERMNO")
fwd_ret = returns.shift(-1).stack()
fwd_ret.name = "RET_T1"

# Get RF
rf = data.set_index("monthid")["RF"]
rf = rf[~rf.index.duplicated()]  # rf indexed by monthid

In [105]:
import math 

# Function to build the portfolio and calculate different performance metrics
def portfolio_build(df_pred: pd.DataFrame, model_name: str, actual_t1_returns: pd.Series, rf_month):
    """
    Params:
        df_pred: DataFrame indexed by [monthid, permno] with column "pred" for predicted T+1 returns
        model_name: Str cosmetic name
        actual_t1_returns: Series indexed by [monthid, permno] with actual T+1 returns
        rf_month: RF returns each month
    """
    actual_t1_returns.name = "ACTUAL_RET_T1"

    data = df_pred.merge(actual_t1_returns, how="left", left_index=True, right_index=True)

    portfolio_ret = []
    # index is monthid + 1, since these returns are for the start of the current period to end of next period, so month ending monthid+1
    index = []
    mth_grp = data.groupby('monthid')
    data['pred_rank'] = mth_grp["pred"].transform(lambda x: pd.qcut(x.rank(method="first"), 10, labels=False) if not np.isnan(x).all() else x)  # if statement in case all NaN
    for monthid, mdata in mth_grp:
      # TODO: can add value weighting of returns here if anyone needs it
      d1 = mdata[mdata['pred_rank'] == 0]['ACTUAL_RET_T1'].mean()
      d10 = mdata[mdata['pred_rank'] == 9]['ACTUAL_RET_T1'].mean()
      portfolio_ret.append(d10 - d1)
      index.append(monthid + 1)

    # Portfolio returns, indexed by monthid of the return end period
    ls_df = pd.DataFrame(portfolio_ret, index=index, columns=["ls_ret"])
    
    # Calculating cumulative returns
    # First we +1 to the returns 
    # Then we do the cumulative product
    ls_df['cumulative_ret']= ls_df['ls_ret'] + 1
    ls_df['cumulative_ret'] = ls_df['cumulative_ret'].cumprod()
    
    # Get mean/std/sharpe ratio for the portfolio
    ls_df = pd.merge(ls_df, rf_month, left_index = True, right_index = True)
    ls_df['ls_sub_rf'] = ls_df['ls_ret'] - ls_df['RF']
    ls_sub_rf_mean = ls_df['ls_sub_rf'].mean()*12
    ls_sub_rf_std = ls_df['ls_sub_rf'].std()*math.sqrt(12)
    sharpe_ratio = ls_sub_rf_mean/ls_sub_rf_std*math.sqrt(12)
    
    print(model_name + " excess return (ann): " + str(ls_sub_rf_mean))
    print(model_name + " volatility (ann): " + str(ls_sub_rf_std))
    print(model_name + " sharpe ratio (ann): " + str(sharpe_ratio))
    print()
    
    return ls_df, ls_sub_rf_mean, ls_sub_rf_std, sharpe_ratio

In [107]:
portfolio_returns, _, _, _ = portfolio_build(preds, "FM", fwd_ret, rf)
portfolio_returns

FM excess return (ann): -0.01899549620801033
FM volatility (ann): 0.2485375333743147
FM sharpe ratio (ann): -0.26475811601223553



,ls_ret,cumulative_ret,RF,ls_sub_rf
13,-0.009464,0.990536,0.0047,-0.014164
14,-0.041892,0.949040,0.0034,-0.045292
15,-0.000175,0.948874,0.0040,-0.004175
16,-0.037805,0.913002,0.0042,-0.042005
17,-0.029695,0.885891,0.0037,-0.033395
...,...,...,...,...
548,-0.033915,0.851257,0.0001,-0.034015
549,0.010868,0.860509,0.0001,0.010768
550,-0.039190,0.826785,0.0001,-0.039290
551,-0.067540,0.770944,0.0001,-0.067640


In [19]:
assets.data

,PERMNO,DATE,CUSIP,SHRCD,EXCHCD,PRC,RET,SHROUT,year,month,...,OIADPQ,PSTKQ,SALEQ,qtrid,DATEFF,SMB,HML,MKTRF,RF,UMD
0,10002,1998-01-30 00:00:00.000000,05978R10,11.0,3.0,25.000,0.020408,4246.0,1998,1,...,2.154,0.00,7.250,273,1998-01-30,-0.0107,-0.0163,0.0015,0.0043,0.0014
1,10016,1998-01-30 00:00:00.000000,81002230,11.0,3.0,13.250,0.009524,13729.0,1998,1,...,5.733,0.00,38.084,273,1998-01-30,-0.0107,-0.0163,0.0015,0.0043,0.0014
2,10019,1998-01-30 00:00:00.000000,44950710,11.0,3.0,16.125,0.040323,8205.0,1998,1,...,3.179,0.00,25.521,273,1998-01-30,-0.0107,-0.0163,0.0015,0.0043,0.0014
3,10025,1998-01-30 00:00:00.000000,00103110,11.0,3.0,33.500,0.085020,7219.0,1998,1,...,10.677,0.00,198.031,271,1998-01-30,-0.0107,-0.0163,0.0015,0.0043,0.0014
4,10026,1998-01-30 00:00:00.000000,46603210,11.0,3.0,14.250,-0.129771,8872.0,1998,1,...,5.293,0.00,62.964,273,1998-01-30,-0.0107,-0.0163,0.0015,0.0043,0.0014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242503,92284,1986-06-30 00:00:00.000000,00794210,11.0,3.0,16.000,0.049180,9064.0,1986,6,...,1.836,1.01,20.347,132,1986-06-30,-0.0096,0.0128,0.0103,0.0052,0.0507
1242504,92567,1986-06-30 00:00:00.000000,19827410,11.0,3.0,14.375,0.017699,24677.0,1986,6,...,2.257,0.00,84.400,132,1986-06-30,-0.0096,0.0128,0.0103,0.0052,0.0507
1242505,92639,1986-06-30 00:00:00.000000,90290110,10.0,3.0,40.875,-0.130319,5486.0,1986,6,...,5.085,0.00,14.231,132,1986-06-30,-0.0096,0.0128,0.0103,0.0052,0.0507
1242506,92655,1986-06-30 00:00:00.000000,91324P10,11.0,3.0,12.625,-0.114035,15167.0,1986,6,...,2.016,0.00,32.164,132,1986-06-30,-0.0096,0.0128,0.0103,0.0052,0.0507


In [20]:
assets.fact

,DATEFF,SMB,HML,MKTRF,RF,UMD,monthid,epq1,dRoa1,RET,PERMNO
0,1975-01-31,0.1114,0.0828,0.1366,0.0058,-0.1382,1,-0.026479,NaN,0.231061,10006
1,1975-01-31,0.1114,0.0828,0.1366,0.0058,-0.1382,1,-0.026479,NaN,0.337349,10102
2,1975-01-31,0.1114,0.0828,0.1366,0.0058,-0.1382,1,-0.026479,NaN,0.333333,10137
3,1975-01-31,0.1114,0.0828,0.1366,0.0058,-0.1382,1,-0.026479,NaN,0.136564,10145
4,1975-01-31,0.1114,0.0828,0.1366,0.0058,-0.1382,1,-0.026479,NaN,0.215447,10161
...,...,...,...,...,...,...,...,...,...,...,...
1242503,2020-12-31,0.0489,-0.0151,0.0463,0.0001,-0.0232,552,0.032500,-0.00936,0.143199,93397
1242504,2020-12-31,0.0489,-0.0151,0.0463,0.0001,-0.0232,552,0.032500,-0.00936,0.109665,93423
1242505,2020-12-31,0.0489,-0.0151,0.0463,0.0001,-0.0232,552,0.032500,-0.00936,0.076239,93426
1242506,2020-12-31,0.0489,-0.0151,0.0463,0.0001,-0.0232,552,0.032500,-0.00936,0.135851,93427


In [41]:
assets.factor_fm

,mean,std_error,tstat
epq1,0.001333,0.000192,6.958548
dRoa1,-0.000118,0.000081,-1.454567


In [22]:
# assets.fact.to_csv('assets.csv', index=False)


In [23]:
assets.gen_factor_betas()

,PERMNO,monthid,alpha,epq1_b,dRoa1_b
0,10002,277,0.017806,-0.197779,-0.307397
1,10006,13,0.006228,0.306043,0.417482
2,10010,205,-0.019804,1.180667,3.135533
3,10016,139,0.004049,0.022531,-0.428573
4,10017,140,0.016547,0.181472,0.304806
...,...,...,...,...,...
7880,93060,229,0.026267,-0.090663,2.079307
7881,93105,265,0.034868,-0.209626,0.280231
7882,93156,181,0.064601,-0.544915,-1.280382
7883,93201,133,-0.068594,2.218845,1.891326


In [24]:
rets = assets.score_linear()  

       PERMNO  C_MKTRF_b  F_MKTRF_b      SMB_b     HML_b      UMD_b
0       10002   0.465144   0.407501   0.464531  2.079725   0.183559
1       10006   1.077032   1.070980   0.115423  0.279345   0.144392
2       10010   3.199802   2.073939   1.276142  0.171664   2.767446
3       10012   2.871593  -6.412176  -8.639767 -7.244200   8.191616
4       10016   0.750906   1.092842   0.827613  1.159553  -0.242292
...       ...        ...        ...        ...       ...        ...
12161   93428   1.493261   1.244205   1.453315 -0.702493   0.064504
12162   93429   0.633694   0.683048   0.055801  0.714426   0.484314
12163   93433   2.329647   4.297205 -14.322695 -4.768507  16.075954
12164   93435   4.735184   1.770314  -1.161409  1.950178  -1.200404
12165   93436   1.761013   1.729920   0.298677 -0.965724  -0.273022

[12166 rows x 6 columns]
     monthid  C_MKTRF_l  F_MKTRF_l     SMB_l     HML_l     UMD_l
0         13   0.135414   0.128623  0.043894  0.076423  0.046004
1         14   0.062810   0.

In [25]:
rets

,monthid,d1_ret,d10_ret,d10-d1_ret,capm_ret,ff_ret
0,13,-0.080361,0.280110,0.360471,0.150545,0.037757
1,25,-0.057487,0.366200,0.423687,0.159300,0.034295
2,37,0.146495,NaN,NaN,0.075653,0.043116
3,49,NaN,NaN,NaN,0.155865,0.017166
4,61,NaN,0.167609,NaN,0.179758,-0.011730
...,...,...,...,...,...,...
56,340,NaN,NaN,NaN,0.150600,0.060548
57,349,0.051829,0.227425,0.175597,0.126400,-0.047315
58,352,0.072478,0.000000,-0.072478,0.223971,0.023880
59,355,0.108694,0.205128,0.096434,0.136267,-0.037128


In [26]:
assets.performance_analytics(rets, ret_col='d10-d1_ret')

,raw_return,sharpe_r,capm_alpha,ff_alpha,information_r
0,0.143798,-0.004407,-0.000881,0.116817,-0.004593
